# Image embeddings: Cats and Dogs

Can the AI identify and differentiate between cats and dogs with only a few examples?

Note that this notebook requires umap and umap.plot; you can install both via `pip3 install umap-learn umap-learn[plot]`

This notebook also requires installing `datasets`.

Sources:
- https://pypi.org/project/imgbeddings/

## Install packages

In [1]:
#pip install imgbeddings

ModuleNotFoundError: No module named 'umap'

In [2]:
from imgbeddings import imgbeddings
from PIL import Image
import logging
import numpy as np
import datasets
from numpy.random import default_rng
import umap
#import umap.plot


logger = logging.getLogger()
logger.setLevel(logging.INFO)


ModuleNotFoundError: No module named 'umap'

The next cell downloads the famous [Cats vs. Dogs dataset](https://huggingface.co/datasets/cats_vs_dogs) which includes about 23 thousand images; the download is about 800MB.

In [ ]:
df = datasets.load_dataset("cats_vs_dogs", split="train", ignore_verifications=True)
df[0:3]

In [ ]:
df

Select 1000 each of random cats (label `0`) and random dogs (label `1`) for a perfectly balanced dataset, as all datasets should be.

In [ ]:
n = 1000

rng = default_rng(42)
cat_idx = rng.choice(np.where(np.array(df["labels"]) == 0)[0], n, replace=False)
dog_idx = rng.choice(np.where(np.array(df["labels"]) == 1)[0], n, replace=False)
idx = np.hstack((cat_idx, dog_idx))
rng.shuffle(idx)

idx[0:5]

In [ ]:
#img_source = [df[int(x)]["image_file_path"] for x in idx]
images = [df[int(x)]["image"] for x in idx]
labels = [df[int(x)]["labels"] for x in idx]

Preview the images we selected:

In [ ]:
# Show some of the loaded images in a grid:
# https://stackoverflow.com/a/65583584


def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

n_rows = 4
n_cols = 8
image_grid([x.resize((64,64)) for x in images[0:(n_rows*n_cols)]], n_rows, n_cols)

In [ ]:
print(labels[0:(n_rows*n_cols)])

Now all the images can be converted to embeddings!

In [ ]:
ibed = imgbeddings()

In [ ]:
embeddings = ibed.to_embeddings(images)

## UMAP unsupervised clustering

umap is a tool that can cluster embeddings (better/more efficient than the popular tSNE) and then visualize the results.

Using cosine similarity works best here.

In [ ]:
!pip install umap-learn

In [ ]:
import umap
umap_catsdogs = umap.UMAP(
    n_neighbors=10, n_components=2, metric="cosine", random_state=42
).fit(embeddings)


In [ ]:
umap.plot.points(umap_catsdogs, labels=np.array(labels))

The two groups are noticeably distinct, although there are several outliers in both cases.

## Cat/Dog Classifier with a simple Logistic Regression

Here's a simple `scikit-learn` binary classifier using the generated embeddings to determine whether an image is a cat or a dog, which is just as simple as any other generic online tutorial.

However, `max_iter` is increased due to the nature of providing raw embeddings vs. tabular data.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

classifier = LogisticRegression(max_iter=500)

In [ ]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(embeddings, labels, test_size=test_size, random_state=42, stratify=labels)

In [ ]:
classifier.fit(x_train, y_train)

In [ ]:
classifier.score(x_test, y_test)

As expected. For comparison, the [Kaggle Competiton](https://www.kaggle.com/competitions/dogs-vs-cats/leaderboard) for this dataset had a best score of 0.989, although with a different/larger validation set, and without the power of modern AI frameworks.

Let's look at misclassified images by finding the indices of where the predictions were incorrect.

In [ ]:
predicted = classifier.predict(x_test)
idx_mismatch = np.where(y_test != predicted)
idx_mismatch


Map those indicies back to the original 2000 image dataset.

In [ ]:
# https://stackoverflow.com/a/51352806
misclass_idx = np.where((embeddings[:, None] == x_test[idx_mismatch]).all(-1).any(-1))
misclass_idx

In [ ]:
n_rows = 1
n_cols = len(list(misclass_idx[0]))
image_grid([images[x].resize((128,128)) for x in list(misclass_idx[0])], n_rows, n_cols)

The first image is interesting: the cage bars "mask" the image of the dog which explain how the model may get confused.

The second image however is shennanigans: putting both a dog and a cat in the same image is cheating IMO.